In [2]:
import json
import stix2
import requests
from typing import List

from mitreattack.stix20 import MitreAttackData

In [3]:
# Generate Local DataBase
infos = requests.get("https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json").json()
with open('enterprise-attack.json', 'w', encoding='utf-8') as f:
    json.dump(infos, f, ensure_ascii=False, indent=4)

In [2]:
def find_mitre_extract_data_by_type(type_data: str, data: List[dict]):
    return [i for i in data if type_data in list(i.values())]


def find_mitre_extract_values_by_key(key_data: str, data: List[dict]):
    return list(set([i[key_data] for i in data if key_data in list(i.keys())]))



def find_mitre_data(data: List[dict], **kwargs):
    selected_data = list()
    external_references = list()
    kill_chain_phases = list()
    for obj in data:
        if "external_references" in list(obj.keys()):
            for element in obj["external_references"]:
                if  "mitre-attack" in list(element.values()):
                    external_references.append(element)

        if "kill_chain_phases" in list(obj.keys()) and "Windows" in obj["x_mitre_platforms"]:
            for element in obj["kill_chain_phases"]:
                if  element["phase_name"] in tatics_deco:
                    kill_chain_phases.append(element)
        
            selected_data.append({
                # "name": obj["name"],
                # "x_mitre_platforms": obj["x_mitre_platforms"],
                "external_references": external_references,
                "kill_chain_phases": kill_chain_phases
            })
            external_references = list()
            kill_chain_phases = list()
    
    return selected_data

In [4]:
mitre_attack_data = MitreAttackData("enterprise-attack.json")

In [ ]:
tactics = mitre_attack_data.get_tactics(remove_revoked_deprecated=True)
data = [json.loads(i.serialize()) for i in tactics]
data

In [ ]:
techniques = mitre_attack_data.get_techniques_by_tactic(
    tactic_shortname='impact', 
    domain='enterprise-attack', 
    remove_revoked_deprecated=False
)

In [5]:
techniques = mitre_attack_data.get_techniques(include_subtechniques=False, remove_revoked_deprecated=True)
data = [json.loads(i.serialize()) for i in techniques]
data[:5]

[{'type': 'attack-pattern',
  'id': 'attack-pattern--01a5a209-b94c-450b-b7f9-946497d91055',
  'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
  'created': '2017-05-31T21:30:44.329Z',
  'modified': '2023-04-07T17:10:13.696Z',
  'name': 'Windows Management Instrumentation',
  'description': 'Adversaries may abuse Windows Management Instrumentation (WMI) to execute malicious commands and payloads. WMI is an administration feature that provides a uniform environment to access Windows system components. The WMI service enables both local and remote access, though the latter is facilitated by [Remote Services](https://attack.mitre.org/techniques/T1021) such as [Distributed Component Object Model](https://attack.mitre.org/techniques/T1021/003) (DCOM) and [Windows Remote Management](https://attack.mitre.org/techniques/T1021/006) (WinRM).(Citation: MSDN WMI) Remote WMI over DCOM operates using port 135, whereas WMI over WinRM operates over port 5985 when using HTTP and 5986 

In [14]:
subtechniques = mitre_attack_data.get_subtechniques_of_technique(technique_stix_id='T1491')
data = [json.loads(i.serialize()) for i in subtechniques]
data

[]